데이터 전처리

In [1]:
import pandas as pd

# 파일 경로 설정
file1_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_컬럼제거.csv"
file2_path = r"C:\Users\m\Downloads\[상권성장지수]증감률통합(서울평균까지).csv"
output_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_결과.csv"

# 데이터 불러오기
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# 컬럼명 통일 (연분기 ↔ 기준_년분기_코드, 상권명 ↔ 상권_코드_명)
df_merged = df1.merge(df2[['기준_년분기_코드', '상권_코드_명', '당월_매출_금액_증감률']],
                      left_on=['연분기', '상권명'],
                      right_on=['기준_년분기_코드', '상권_코드_명'],
                      how='left')

# 불필요한 컬럼 삭제
df_merged.drop(columns=['기준_년분기_코드', '상권_코드_명'], inplace=True)

# 상권성장상태 컬럼 생성
def classify_growth(rate):
    if rate >= 5:
        return "성장"
    elif rate <= -5:
        return "쇠퇴"
    else:
        return "정체"

df_merged['상권성장상태'] = df_merged['당월_매출_금액_증감률'].apply(classify_growth)

# 결과 저장
df_merged.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"처리 완료: {output_path}")

처리 완료: C:\Users\m\Downloads\서울시_상권분석_서비스_결과.csv


In [2]:
import pandas as pd

# 파일 경로 설정
file_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_결과.csv"
output_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_결과_수정.csv"

# 데이터 불러오기
df = pd.read_csv(file_path)

# '연분기' 값이 20191인 행 제거
df_filtered = df[df['연분기'] != 20191]

# 수정된 데이터 저장
df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"처리 완료: {output_path}")

처리 완료: C:\Users\m\Downloads\서울시_상권분석_서비스_결과_수정.csv


In [3]:
import pandas as pd

# 파일 경로 설정
file_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_결과_수정.csv"
output_path = r"C:\Users\m\Downloads\서울시_상권분석_서비스_결과_최종.csv"

# 데이터 불러오기
df = pd.read_csv(file_path)

# '당월_매출_금액_증감률' 컬럼의 값이 NaN인 행 제거
df_cleaned = df.dropna(subset=['당월_매출_금액_증감률'])

# 수정된 데이터 저장
df_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"처리 완료: {output_path}")

처리 완료: C:\Users\m\Downloads\서울시_상권분석_서비스_결과_최종.csv


안정성 관련 변수 생성 및 정의.

In [5]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성 데이터.csv"
df = pd.read_csv(file_path)

# ✅ 안정성 변수 정의
df["개폐업_비율"] = df["폐업_점포_수"] / (df["개업_점포_수"] + 1)  
df["영업_개월_차이"] = df["운영_영업_개월_평균"] - df["폐업_영업_개월_평균"]
df["프랜차이즈_비율"] = df["프랜차이즈_점포_수"] / df["점포_수"]
df["점포_증감률"] = (df["점포_수"] - df["점포_수"].shift(1)) / df["점포_수"].shift(1) * 100

# ✅ 성장, 쇠퇴 상권에서 안정성 변수들의 최솟값/최댓값 찾기
growth_min = df[df["상권성장상태"] == "성장"][["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률"]].min()
decline_max = df[df["상권성장상태"] == "쇠퇴"][["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률"]].max()

# ✅ 상권 안정성 분류 함수
def classify_stability(row):
    if (row["개폐업_비율"] >= decline_max["개폐업_비율"]) or \
       (row["영업_개월_차이"] <= decline_max["영업_개월_차이"]) or \
       (row["프랜차이즈_비율"] <= decline_max["프랜차이즈_비율"]) or \
       (row["점포_증감률"] <= decline_max["점포_증감률"]):
        return "불안정"

    elif (row["개폐업_비율"] <= growth_min["개폐업_비율"]) and \
         (row["영업_개월_차이"] >= growth_min["영업_개월_차이"]) and \
         (row["프랜차이즈_비율"] >= growth_min["프랜차이즈_비율"]) and \
         (row["점포_증감률"] >= growth_min["점포_증감률"]):
        return "안정"

    else:
        return "중립"

# ✅ 상권 안정성 분류 적용
df["상권_안정성"] = df.apply(classify_stability, axis=1)

# ✅ 결과 저장
output_path = r"C:\Users\m\Downloads\매출-안정성 데이터_분류.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"처리 완료: {output_path}")

처리 완료: C:\Users\m\Downloads\매출-안정성 데이터_분류.csv


In [7]:
import pandas as pd

# 데이터 로드
df = pd.read_csv(r"C:\Users\m\Downloads\매출-안정성 데이터.csv")

# ✅ 안정성 변수 정의 (최근 4개 분기 평균 이용)
df["개폐업_비율"] = df.groupby(["상권명", "업종명"])['폐업_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) / (df.groupby(["상권명", "업종명"])['개업_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) + 1)
df["영업_개월_차이"] = df.groupby(["상권명", "업종명"])['운영_영업_개월_평균'].transform(lambda x: x.rolling(4, min_periods=1).mean()) - df.groupby(["상권명", "업종명"])['폐업_영업_개월_평균'].transform(lambda x: x.rolling(4, min_periods=1).mean())
df["프랜차이즈_비율"] = df.groupby(["상권명", "업종명"])['프랜차이즈_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) / df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean())
df["점포_증감률"] = (df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()).diff()) / df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()).shift(1) * 100

# ✅ 안정성 지표 범위 설정 (상권성장상태 기반)
성장_최솟값 = df[df["상권성장상태"] == "성장"][["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률"]].min()
쇠퇴_최댓값 = df[df["상권성장상태"] == "쇠퇴"][["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률"]].max()

# ✅ 안정성 지표에 따라 성장, 정체, 쇠퇴 재분류
def classify_growth(row):
    if (row["개폐업_비율"] >= 쇠퇴_최댓값["개폐업_비율"]) or \
       (row["영업_개월_차이"] <= 쇠퇴_최댓값["영업_개월_차이"]) or \
       (row["프랜차이즈_비율"] <= 쇠퇴_최댓값["프랜차이즈_비율"]) or \
       (row["점포_증감률"] <= 쇠퇴_최댓값["점포_증감률"]):
        return "쇠퇴"
    elif (row["개폐업_비율"] <= 성장_최솟값["개폐업_비율"]) and \
         (row["영업_개월_차이"] >= 성장_최솟값["영업_개월_차이"]) and \
         (row["프랜차이즈_비율"] >= 성장_최솟값["프랜차이즈_비율"]) and \
         (row["점포_증감률"] >= 성장_최솟값["점포_증감률"]):
        return "성장"
    else:
        return "정체"

df["재분류_상권성장상태"] = df.apply(classify_growth, axis=1)

# ✅ 결과 저장
df.to_csv(r"C:\Users\m\Downloads\매출-안정성 데이터_처리완료.csv", index=False)

In [8]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성 데이터.csv"
df = pd.read_csv(file_path)

# ✅ 안정성 변수 정의 (최근 4개 분기 평균 이용)
df["개폐업_비율"] = df.groupby(["상권명", "업종명"])['폐업_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) / (df.groupby(["상권명", "업종명"])['개업_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) + 1)
df["영업_개월_차이"] = df.groupby(["상권명", "업종명"])['운영_영업_개월_평균'].transform(lambda x: x.rolling(4, min_periods=1).mean()) - df.groupby(["상권명", "업종명"])['폐업_영업_개월_평균'].transform(lambda x: x.rolling(4, min_periods=1).mean())
df["프랜차이즈_비율"] = df.groupby(["상권명", "업종명"])['프랜차이즈_점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()) / df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean())
df["점포_증감률"] = (df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()).diff()) / df.groupby(["상권명", "업종명"])['점포_수'].transform(lambda x: x.rolling(4, min_periods=1).mean()).shift(1) * 100

# 데이터 저장
output_path = r"C:\Users\m\Downloads\매출-안정성_계산완료.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"파일 저장 완료: {output_path}")

파일 저장 완료: C:\Users\m\Downloads\매출-안정성_계산완료.csv


상관관계 분석

In [9]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성_계산완료.csv"
df = pd.read_csv(file_path)

# ✅ 20241 데이터 필터링
df_20241 = df[df["연분기"] == 20241]

# ✅ 상관관계 분석 대상 컬럼 선택
correlation_columns = ["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률", "당월_매출_금액_증감률"]

# ✅ 해당 컬럼들 간의 상관관계 계산
correlation_matrix = df_20241[correlation_columns].corr()

# ✅ 결과 출력
print(correlation_matrix)

                개폐업_비율  영업_개월_차이  프랜차이즈_비율    점포_증감률  당월_매출_금액_증감률
개폐업_비율        1.000000 -0.083057  0.067180  0.270814     -0.009518
영업_개월_차이     -0.083057  1.000000 -0.015176 -0.013317     -0.039737
프랜차이즈_비율      0.067180 -0.015176  1.000000 -0.009188     -0.003199
점포_증감률        0.270814 -0.013317 -0.009188  1.000000     -0.004570
당월_매출_금액_증감률 -0.009518 -0.039737 -0.003199 -0.004570      1.000000


In [10]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성_계산완료.csv"
df = pd.read_csv(file_path)

# ✅ 상관관계 분석 대상 컬럼 선택
correlation_columns = ["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률", "당월_매출_금액_증감률"]

# ✅ 상권별 상관관계 계산
correlation_results = {}

for 상권명, group in df.groupby("상권명"):
    correlation_matrix = group[correlation_columns].corr()
    correlation_results[상권명] = correlation_matrix["당월_매출_금액_증감률"].drop("당월_매출_금액_증감률")

# ✅ 결과 출력
for 상권명, correlation in correlation_results.items():
    print(f"\n📍 {상권명} 상권의 상관관계")
    print(correlation)


📍 4.19민주묘지역 2번 상권의 상관관계
개폐업_비율      0.051768
영업_개월_차이    0.011692
프랜차이즈_비율    0.009847
점포_증감률     -0.014310
Name: 당월_매출_금액_증감률, dtype: float64

📍 63빌딩 상권의 상관관계
개폐업_비율     -0.014457
영업_개월_차이    0.053988
프랜차이즈_비율    0.003077
점포_증감률      0.048371
Name: 당월_매출_금액_증감률, dtype: float64

📍 DMC(디지털미디어시티) 상권의 상관관계
개폐업_비율      0.014385
영업_개월_차이   -0.068286
프랜차이즈_비율    0.003697
점포_증감률     -0.021161
Name: 당월_매출_금액_증감률, dtype: float64

📍 GS강동자이아파트 상권의 상관관계
개폐업_비율     -0.032471
영업_개월_차이   -0.268431
프랜차이즈_비율   -0.006490
점포_증감률     -0.022350
Name: 당월_매출_금액_증감률, dtype: float64

📍 G타워 상권의 상관관계
개폐업_비율      0.029970
영업_개월_차이    0.203695
프랜차이즈_비율    0.004721
점포_증감률     -0.023318
Name: 당월_매출_금액_증감률, dtype: float64

📍 KB국민은행 망원동지점 상권의 상관관계
개폐업_비율     -0.019281
영업_개월_차이    0.049214
프랜차이즈_비율   -0.010944
점포_증감률      0.006777
Name: 당월_매출_금액_증감률, dtype: float64

📍 KB국민은행 서강지점 상권의 상관관계
개폐업_비율      0.015590
영업_개월_차이   -0.065024
프랜차이즈_비율   -0.036183
점포_증감률     -0.010400
Name: 당월_매출_금액_증감률, dtype: float64

📍 KB국민은행 종암

In [11]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성_계산완료.csv"
df = pd.read_csv(file_path)

# ✅ 20242 분기 데이터 필터링
df_20242 = df[df["연분기"] == 20242]

# ✅ 상관관계 분석 대상 컬럼 선택
correlation_columns = ["개폐업_비율", "영업_개월_차이", "프랜차이즈_비율", "점포_증감률", "당월_매출_금액_증감률"]

# ✅ 상권별 상관관계 계산
correlation_results = []

for 상권명, group in df_20242.groupby("상권명"):
    correlation_matrix = group[correlation_columns].corr()
    correlation_values = correlation_matrix["당월_매출_금액_증감률"].drop("당월_매출_금액_증감률")
    correlation_results.append(correlation_values)

# ✅ 상관계수 평균 계산
correlation_mean = pd.DataFrame(correlation_results).mean()

# ✅ 결과 출력
print("\n📌 20242 분기 상권별 상관관계의 평균")
print(correlation_mean)


📌 20242 분기 상권별 상관관계의 평균
개폐업_비율     NaN
영업_개월_차이   NaN
프랜차이즈_비율   NaN
점포_증감률     NaN
dtype: float64


In [16]:
import pandas as pd

# 데이터 불러오기
file_path = r"C:\Users\m\Downloads\매출-안정성_계산완료.csv"
df = pd.read_csv(file_path)

# ✅ 컬럼명 확인 및 공백 제거
df.columns = df.columns.str.strip()  # 혹시 모를 공백 제거
print("🔍 데이터 컬럼명 확인:", df.columns)

# ✅ 20242 분기 데이터 필터링
df_20242 = df[df["연분기"] == 20242]

# ✅ 1. 컬럼명 공백 제거
df_20242.rename(columns=lambda x: x.strip(), inplace=True)

# ✅ 2. 숫자로 변환 가능한 컬럼만 선택
numeric_columns = df_20242.select_dtypes(include=["number"]).columns
df_20242 = df_20242[numeric_columns]

# ✅ 3. 상관관계 계산 (수정된 코드)
def calculate_corr(group):
    if len(group) > 1:
        corr_matrix = group.corr()
        if "당월_매출_금액_증감률" in corr_matrix:
            return corr_matrix.loc["당월_매출_금액_증감률"].drop("당월_매출_금액_증감률", errors="ignore")
    return pd.Series(index=numeric_columns[:-1], dtype="float64")  # 빈 값 반환

correlation_results = df_20242.groupby("상권명").apply(calculate_corr)

# ✅ 4. NaN 값 제외 후 평균 계산
correlation_mean = correlation_results.mean(skipna=True)

# 결과 출력
print("\n📌 20242 분기 상권별 상관관계의 평균")
print(correlation_mean)

🔍 데이터 컬럼명 확인: Index(['연분기', '상권명', '업종명', '점포_수', '유사_업종_점포_수', '개업율', '개업_점포_수', '폐업률',
       '폐업_점포_수', '프랜차이즈_점포_수', '자치구명', '운영_영업_개월_평균', '폐업_영업_개월_평균',
       '당월_매출_금액_증감률', '상권성장상태', '개폐업_비율', '영업_개월_차이', '프랜차이즈_비율', '점포_증감률'],
      dtype='object')


C:\Users\m\AppData\Local\Temp\ipykernel_20176\2973739088.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20242.rename(columns=lambda x: x.strip(), inplace=True)


KeyError: '상권명'

다시 전처리

In [17]:
import pandas as pd

# ✅ 데이터 불러오기
file_path = "C:\\Users\\m\\Downloads\\매출-안정성 데이터.csv"
df = pd.read_csv(file_path)

# ✅ 📌 1. 개폐업 변화량 (개업 - 폐업)
df["개폐업_변화량"] = df["개업_점포_수"] - df["폐업_점포_수"]

# ✅ 📌 2. 영업 지속 기간 차이 (운영 영업 개월 - 폐업 영업 개월)
df["영업_지속_기간_차이"] = df["운영_영업_개월_평균"] - df["폐업_영업_개월_평균"]

# ✅ 📌 3. 점포 증감량 (현재 점포 수 - 유사 업종 점포 수)
df["점포_증감량"] = df["점포_수"] - df["유사_업종_점포_수"]

# ✅ 📌 4. 프랜차이즈 안정성 점수 (25:50:25 비율 적용)
q1 = df["프랜차이즈_점포_수"].quantile(0.25)  # 하위 25% (Q1)
q2 = df["프랜차이즈_점포_수"].quantile(0.75)  # 상위 25% (Q3)

def franchise_stability_score(franchise_count):
    if franchise_count == 0:
        return 0  # 프랜차이즈 없음
    elif franchise_count <= q1:
        return 1  # 하위 25%
    elif franchise_count <= q2:
        return 2  # 중간 50%
    else:
        return 3  # 상위 25%

df["프랜차이즈_안정성_점수"] = df["프랜차이즈_점포_수"].apply(franchise_stability_score)

# ✅ 📌 5. 불필요한 컬럼 제거 (겹치는 정보)
drop_columns = [
    "개업_점포_수", "폐업_점포_수", "운영_영업_개월_평균", "폐업_영업_개월_평균", 
    "점포_수", "유사_업종_점포_수", "프랜차이즈_점포_수"
]
df.drop(columns=drop_columns, inplace=True)

# ✅ 새로운 데이터 저장
output_path = "C:\\Users\\m\\Downloads\\매출-안정성 데이터_추가.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

# ✅ 처리 완료 메시지
print(f"새로운 데이터 저장 완료: {output_path}")

새로운 데이터 저장 완료: C:\Users\m\Downloads\매출-안정성 데이터_추가.csv


In [18]:
import pandas as pd

# ✅ 데이터 불러오기
file_path = "C:\\Users\\m\\Downloads\\매출-안정성 데이터.csv"
df = pd.read_csv(file_path)

# ✅ 📌 1. 개폐업 변화량 (개업 - 폐업)
df["개폐업_변화량"] = df["개업_점포_수"] - df["폐업_점포_수"]

# ✅ 📌 2. 영업 지속 기간 차이 (운영 영업 개월 - 폐업 영업 개월)
df["영업_지속_기간_차이"] = df["운영_영업_개월_평균"] - df["폐업_영업_개월_평균"]

# ✅ 📌 3. 점포 증감량 (현재 점포 수 - 유사 업종 점포 수)
df["점포_증감량"] = df["점포_수"] - df["유사_업종_점포_수"]

# ✅ 📌 4. 프랜차이즈 안정성 점수 (25:50:25 비율 적용)
q1 = df["프랜차이즈_점포_수"].quantile(0.25)  # 하위 25% (Q1)
q2 = df["프랜차이즈_점포_수"].quantile(0.75)  # 상위 25% (Q3)

def franchise_stability_score(franchise_count):
    if franchise_count == 0:
        return 0  # 프랜차이즈 없음
    elif franchise_count <= q1:
        return 1  # 하위 25%
    elif franchise_count <= q2:
        return 2  # 중간 50%
    else:
        return 3  # 상위 25%

df["프랜차이즈_안정성_점수"] = df["프랜차이즈_점포_수"].apply(franchise_stability_score)

# ✅ 새로운 데이터 저장
output_path = "C:\\Users\\m\\Downloads\\매출-안정성 데이터_추가.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

# ✅ 처리 완료 메시지
print(f"새로운 데이터 저장 완료: {output_path}")

새로운 데이터 저장 완료: C:\Users\m\Downloads\매출-안정성 데이터_추가.csv
